In [ ]:
##############################
# NOTE: This will fail for large dataset processing or complex model evaluation
# Use the command below to run it in the background 
##############################
# Source: https://www.maksimeren.com/post/screen-and-jupyter-a-way-to-run-long-notebooks-headless/
# jupyter nbconvert --to notebook --execute used_car_price_analysis.template.ipynb --output=used_car_price_analysis.out.ipynb --ExecutePreprocessor.timeout=-1


##############################
# Library Imports
##############################
import copy
import json
import itertools
import math
import os
import io
from datetime import datetime
from random import randint

import matplotlib.pyplot as plt

# Frameworks
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns


# Plotly
import plotly.graph_objects as go
import plotly.io as pio

# scikit-learn
from sklearn import metrics
from sklearn import set_config
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel, RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer, make_column_selector, TransformedTargetRegressor, ColumnTransformer
from sklearn.inspection import permutation_importance
from category_encoders import TargetEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB

# xgboost
import xgboost as xgb
from xgboost import XGBClassifier

##############################
# Runtime Settings
##############################
set_config(display="diagram")
import warnings
warnings.filterwarnings('ignore')

##############################
# Flow control + Constants
##############################
DEV_MODE = False
USE_TARGET_ENCODER_FOR_HIGH_CARDINALTY = True
MODEL1_ACTIVE = True
MODEL2_ACTIVE = True
MODEL3_ACTIVE = True
MODEL4_ACTIVE = False
IMAGE_DIR_SUFFIX = ''


# Constants
RESULT_DIR = './analysis_results'
if DEV_MODE:
    RESULT_DIR = RESULT_DIR + '_dev'
RESULT_FILE_PREFIX = RESULT_DIR +'/module_11_01.'
STEP01_DATA_UNDERSTANDING = 'step01.data_understanding.'
STEP02_DATA_PREPARATION = 'step02.data_preparation.'
STEP03_MODELING = 'step03.modeling.'
STEP04_EVALUATION = 'step04.evaluation.'

TARGET_FIELD = 'price'
ALPHA_START = 0.00001
ALPHA_END = 100
ALPHA_STEP = (ALPHA_END - ALPHA_START)/10
ALPHA_RANGE = np.arange(ALPHA_START, ALPHA_END, ALPHA_STEP)
IGNORE_FEATURES = ['VIN', 'id', 'model', 'paint_color']
CONFIG_NODE_KEY = 'config'
Q3_PERCENT_DEFAULT = 75
Q1_PERCENT_DEFAULT = 25

if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

##############################
# Common global methods
##############################
def writeString2File(string2Write, path, print2Screen = True):
    if print2Screen:
        print(string2Write)
    with open(path, "w") as text_file:
        text_file.write(str(string2Write))

def dataFrame2Image(inputDf, path, baseHeight=480, height_per_row=20, char_limit=30, height_padding=16.5, baseWidth=1200):
    total_height = 0 + baseHeight
    for x in range(inputDf.shape[0]):
        total_height += height_per_row
    for y in range(inputDf.shape[1]):
        if len(str(inputDf.iloc[x][y])) > char_limit:
            total_height += height_padding

    height = total_height
    width = baseWidth
    layout = {
        'height': height, 
        'width': width,
        'margin': {'r': 1, 'l': 1, 't': 1, 'b': 1}
    }
    
    fig = go.Figure(data=[
            go.Table(
                header=dict(values=list(inputDf.columns), fill_color='paleturquoise', align='left'),
                cells=dict(values=inputDf.transpose().values.tolist(), fill_color='lavender', align='left')
            )], layout=layout)
    fig.show()
    
    pio.write_image(fig, path, scale=1, width=width, height=total_height)

# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

<b>Business Understanding:</b> Determine which combination of numeric and categorical features and corresponding categorical values most contribute to driving price increase for used cars. Rank the features and values based on weight to give the used car dealers a simple way to prioritize used cars to display

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
rawDf = pd.read_csv('./data/vehicles.csv')

def writeDataFrame2Excel(inputDf, name):
    with pd.ExcelWriter(RESULT_FILE_PREFIX + name + 'data.frame.xlsx') as writer:
        inputDf.to_excel(writer)
    
def writeDataFrameDetails(data, fileNameSuffix, titleSuffix='', width=480, height=400, bins=100):
    buffer = io.StringIO()
    data.info(verbose=True, buf=buffer, show_counts=True)
    writeString2File(buffer.getvalue(), RESULT_FILE_PREFIX + fileNameSuffix + 'data.info.txt')
    writeString2File(data.describe(), RESULT_FILE_PREFIX + fileNameSuffix + 'data.describe.txt')
    
    plotDistribution4Fields = data.select_dtypes(include=[np.number]).columns.values
    graphsPerRow = len(plotDistribution4Fields)
    if graphsPerRow > 0:
        widthInches = width * graphsPerRow/100
        heightInches = height * 2/100
        plt.clf()
        plt.figure()
        fig, ax = plt.subplots(2, len(plotDistribution4Fields), figsize=(widthInches, heightInches))
        subPlotRow = 0
        subPlotCol = 0
        cm = sns.color_palette("plasma",bins)
        for field in plotDistribution4Fields:
            axSub1 = sns.boxplot(ax=ax[0,subPlotCol], data=data[[field]], palette='pastel')
            axSub2 = sns.histplot(ax=ax[1,subPlotCol], data=data, x=field, bins=bins)
            for bin_,i in zip(axSub2.patches,cm):
                bin_.set_facecolor(i)
            subPlotCol += 1

        fig.suptitle('Data Distribution' + titleSuffix)
        plt.savefig(RESULT_FILE_PREFIX + fileNameSuffix + 'data.distribution.png')
        
    

# Reduce data set 
if DEV_MODE == True:
    rawDf  = rawDf.dropna()
    rawDf = rawDf.head(10000)

writeDataFrameDetails(data=rawDf, fileNameSuffix=STEP01_DATA_UNDERSTANDING, titleSuffix=': Raw Data')

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [ ]:
fixedCounter = {}
def showBarPlot(request, data, xColumn, yColumn, colorColumn, path, xLogScale=False, yLogScale=False, width=1200, height=600, dataLabelRotation=0, moveLegendBotttom=False):
    labels = request['labels']
    xAxesLabel = labels[xColumn]
    if xLogScale:
        xAxesLabel += ' (Log Scale)'
    yAxesLabel = labels[yColumn]
    if yLogScale:
        yAxesLabel += ' (Log Scale)'
    title = yAxesLabel + ' vs ' + xAxesLabel
    barWidthPixels = 80
    if data.shape[0] > 0:
        palette='pastel'
        barWidthInches = barWidthPixels / 100
        # Width of graph is number of rows * bar_width 
        widthInches = math.ceil(width / 100)
        heightInches = math.ceil(height / 100)
        legendLocation='lower left'
        bboxAnchor=(0, -0.5)
        plt.clf()
        plt.figure()
        axSub = sns.barplot(data=data, x=xColumn, y=yColumn, hue=colorColumn, 
                         palette=palette, width=barWidthInches)
        axSub.set_title(title)
        axSub.set(xlabel=xAxesLabel, ylabel=yAxesLabel)
        for bars_group in axSub.containers: 
            axSub.bar_label(bars_group, padding=3, fontsize=8, rotation=dataLabelRotation, label_type='center')
        if yLogScale:
            plt.yscale('log')
        if xLogScale:
            plt.xscale('log')
        if moveLegendBotttom:
            sns.move_legend(axSub, legendLocation, bbox_to_anchor=bboxAnchor)
        axSub.figure.set_size_inches(widthInches, heightInches)
        
        plt.savefig(path, bbox_inches='tight')
        plt.show()
        
def showDataPreparationBarPlot(request, data, xColumn, yColumn, colorColumn, fileSuffix, xLogScale=False, yLogScale=False, width=1200, height=600):
    filePrefix = RESULT_FILE_PREFIX + STEP02_DATA_PREPARATION
    showBarPlot(request=request, data=data, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                path=filePrefix + fileSuffix, xLogScale=xLogScale, yLogScale=yLogScale, width=width, height=height, dataLabelRotation=90, moveLegendBotttom=True)

def displayStats(request, dataReportDf, restrictToStages = []):
    suffix = ''
    if(len(restrictToStages) > 0):
        suffix += '-'.join(restrictToStages) + '.'
    labels = request['labels']
    filteredDf = dataReportDf
    if len(restrictToStages) > 0:
        filteredDf = dataReportDf.loc[dataReportDf['stage'].isin(restrictToStages)]

    columnNames = list(filteredDf['column'].value_counts().index)
    if 'excludeColumnsFromReport' in request:
        excludeColumns = request['excludeColumnsFromReport']
        for k in excludeColumns:
            if k in columnNames:
                columnNames.remove(k)
    filteredDf = filteredDf.loc[filteredDf['column'].isin(columnNames)]
    
    xColumn = 'column'
    colorColumn = 'stage'
    
    yColumn = 'row_count'
    data = filteredDf.loc[filteredDf['column'] == columnNames[0]]
    showDataPreparationBarPlot(request=request, data=data, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                               fileSuffix=suffix + 'row_count.png', height=800)
    
    yColumn = 'na_value_count'
    data = filteredDf[filteredDf[yColumn] != 0]
    showDataPreparationBarPlot(request=request, data=data, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                               fileSuffix=suffix + 'missing_count.png', yLogScale=True, height=800, width=2400)

    yColumn = 'na_value_pct'
    data = filteredDf[filteredDf[yColumn] != 0]
    showDataPreparationBarPlot(request=request, data=data, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                               fileSuffix=suffix + 'missing_percentage.png', yLogScale=True, height=800, width=2400)

    yColumn = 'unq_value_count'
    data = filteredDf.loc[filteredDf['data_type'] == 'object']
    showDataPreparationBarPlot(request=request, data=data, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                               fileSuffix=suffix + 'unique_values.png', yLogScale=True, height=800, width=2400)

def getMostCommonValues(inputDf, groupByFields, columns):
    finalList = groupByFields
    cleanedDf = inputDf.dropna(subset = finalList)
    responseDf = cleanedDf.groupby(groupByFields)[columns].agg(pd.Series.mode).reset_index()
    return responseDf
    
def getType(row, orginalDf):
    return str(orginalDf.dtypes[row['column']])
    
def getUniqueCounts(row, orginalDf):
    return len(orginalDf[row['column']].unique())

def fillEmptyValues(row, lookupTable, groupByFields, fillFields):
    global fixedCounter
    key = ''
    for gf in groupByFields:
        key += str(row[gf])
    if key in lookupTable:
        match = lookupTable[key]
        for ff in fillFields:
            if pd.isna(row[ff]):
                if ff not in fixedCounter:
                    fixedCounter[ff] = 0
                fixedCounter[ff] += 1
                row[ff] = match[ff]
    return row

def findAndFixMissingUsingMode(inputDf, groupByFields, fillFields):
    lookupTable = getMostCommonValues(inputDf, groupByFields, fillFields)
    cache = {}
    for index, row in lookupTable.iterrows():
        key = ''
        for gf in groupByFields:
            key += str(row[gf])
        node = {}
        cache[key] = node
        for f in fillFields:
            node[f] = None
            val = row[f]
            if isinstance(val,np.ndarray):
                if len(val) > 0:
                    val = val[0]
            else:
                node[f] = val
    finalDf = inputDf.apply(fillEmptyValues, lookupTable = cache, groupByFields = groupByFields, fillFields = fillFields, axis = 1)
    return finalDf

def addMetadataStatistics(stage, inputDf, 
                          dataReportDf = pd.DataFrame(columns = [
                              'stage', 
                              'row_count', 
                              'column', 
                              'data_type', 
                              'na_value_count', 
                              'na_value_pct', 
                              'unq_value_count'])):
    naSeries = inputDf.isna().sum()
    numRows = inputDf.shape[0]
    
    metaDf = pd.DataFrame()
    metaDf['column'] = naSeries.index
    metaDf['data_type'] = metaDf.apply(getType, orginalDf=inputDf, axis=1)
    metaDf['na_value_count']= naSeries.values
    metaDf['na_value_pct']= metaDf['na_value_count'] * 100 / numRows
    metaDf['unq_value_count'] = metaDf.apply(getUniqueCounts, orginalDf=inputDf, axis=1)
    metaDf['stage'] = stage
    metaDf['row_count'] = numRows

    for index, row in metaDf.iterrows():
        dataReportDf.loc[dataReportDf.shape[0]] = row
    return dataReportDf

def doDrop(operationNode, inputDf, dataReportDf):
    configNode = operationNode[CONFIG_NODE_KEY]
    keys = []
    for k in configNode:
        keys.append(k)
        inputDf = inputDf.drop(k, axis=1)
    return inputDf, dataReportDf

def doDropNa(operationNode, inputDf, dataReportDf):
    configNode = operationNode[CONFIG_NODE_KEY]
    keys = []
    for k in configNode:
        keys.append(k)
        inputDf = inputDf.dropna(axis=0, subset=[k])
    dataReportDf = addMetadataStatistics(f'dropna[{keys}]', inputDf, dataReportDf)
    return inputDf, dataReportDf
    
def doFillNa(operationNode, inputDf, dataReportDf):
    configNode = operationNode[CONFIG_NODE_KEY]
    for k in configNode:
        node = configNode[k]
        value2Fill = node['value']
        inputDf[k].fillna(value2Fill, inplace=True)
    return inputDf, dataReportDf

def doFillUsingMode(operationNode, inputDf, dataReportDf):
    outputDf = inputDf
    configNode = operationNode[CONFIG_NODE_KEY]
    for k in configNode:
        node = configNode[k]
        lookupFields = node['lookupFields']
        fillFields = node['fillFields']
        outputDf = findAndFixMissingUsingMode(outputDf, lookupFields, fillFields)
        dataReportDf = addMetadataStatistics(f'fill{fillFields} = lookup{lookupFields}', outputDf, dataReportDf)
    return outputDf, dataReportDf

def doQueryFiltering(operationNode, inputDf, dataReportDf):
    outputDf = inputDf
    configNode = operationNode[CONFIG_NODE_KEY]
    for k in configNode:
        node = configNode[k]
        query = node['query']
        outputDf = outputDf.query(query)
        dataReportDf = addMetadataStatistics(f'query({k})', outputDf, dataReportDf)
    return outputDf, dataReportDf
    
def doIqrFiltering(operationNode, inputDf, dataReportDf):
    outputDf = inputDf
    # NOTE: have to calculate quartiles all at once otherwise quartiles shift as we cull the dataset
    quartileRanges = {}
    outputDf = inputDf
    configNode = operationNode[CONFIG_NODE_KEY]
    
    for k in configNode:
        node = configNode[k]
        q3Threshold = 0.75
        q1Threshold = 0.25
        if 'q3%' in node:
            q3Threshold = float(node['q3%']) / 100
        if 'q1%' in node:
            q1Threshold = float(node['q1%']) / 100
        Q3 = np.quantile(outputDf[k], q3Threshold)
        Q1 = np.quantile(outputDf[k], q1Threshold)
        IQR = Q3 - Q1
        lowerRange = Q1 - 1.5 * IQR
        upperRange = Q3 + 1.5 * IQR
        quartileRanges[k]= {
            'q3%': q3Threshold * 100,
            'q1%': q1Threshold * 100,
            'lowerRange': lowerRange,
            'upperRange': upperRange
        }

    # Now do all quartiles together
    for k in quartileRanges:
        node = quartileRanges[k]
        q3 = node['q3%']
        q1 = node['q1%']
        lowerRange = node['lowerRange']
        upperRange = node['upperRange']
        iqrStageDisplayName = f'iqrFilter({k}, {lowerRange} [{q1}%], {upperRange}[{q3}%])' 
        outputDf = outputDf.loc[(outputDf[k] >= lowerRange) & (outputDf[k] <= upperRange)]
        dataReportDf = addMetadataStatistics(iqrStageDisplayName, outputDf, dataReportDf)
    return outputDf, dataReportDf

def doToLowerCase(operationNode, inputDf, dataReportDf):
    configNode = operationNode[CONFIG_NODE_KEY]
    for k in configNode:
        node = configNode[k]
        targetField = k;
        if 'field' in node:
            targetField = node['field']
        inputDf[targetField] = inputDf[k].str.lower()
        if 'removeSpaces' in node:
            if node['removeSpaces'] == True:
                inputDf[targetField] = inputDf[targetField].str.replace(" ", "")
    return inputDf, dataReportDf

def preProcessData(request, inputDf, dataReportDf):
    # Copy once then use same for all transforms
    responseDf = inputDf.copy()
    operations = request['operations']
    for node in operations:
        operationName = node['operation']
        print(f'Processing: {operationName}')
        if operationName == 'drop':
            responseDf, dataReportDf = doDrop(node, responseDf, dataReportDf)
        elif operationName == 'dropna':
            responseDf, dataReportDf = doDropNa(node, responseDf, dataReportDf)
        elif operationName == 'fillna':
            responseDf, dataReportDf = doFillNa(node, responseDf, dataReportDf)
        elif operationName == 'queryFilter':
            responseDf, dataReportDf = doQueryFiltering(node, responseDf, dataReportDf)
        elif operationName == 'iqr':
            responseDf, dataReportDf = doIqrFiltering(node, responseDf, dataReportDf)
        elif operationName == 'toLowerCase':
            responseDf, dataReportDf = doToLowerCase(node, responseDf, dataReportDf)
        elif operationName == 'fillUsingMode':
            responseDf, dataReportDf = doFillUsingMode(node, responseDf, dataReportDf)
        else:
            raise Exception(f"Unsupported operation: {operation}")
    
    return responseDf, dataReportDf


dataPreparationRequest = {
    'excludeColumnsFromReport': ['VIN', 'id', 'cleaned_model'],
    'operations': [
         {
                'operation': 'drop',
                'config' : {
                    'VIN': {},
                    'id': {},
                    'paint_color': {},
                }
         },
         {
            'operation': 'toLowerCase',
            'config' : {
                'state': {},
                'region': {},
                'condition': {},
                'cylinders': {},
                'fuel': {},
                'title_status': {},
                'transmission': {},
                'drive': {},
                'manufacturer': {},
                'size': {},
                'type': {},
                'model': {
                    'field': 'cleaned_model',
                    'removeSpaces': True
                }
            }
        },
        {
            'operation': 'dropna',
            'config': {
                'price': {},
                'model': {}, 
                'manufacturer': {},
                'odometer': {},
                'year': {}
            }
        },
        {
            'operation': 'iqr',
            'config': {
                'price': {
                    'q3%': Q3_PERCENT_DEFAULT,
                    'q1%': Q1_PERCENT_DEFAULT
                },
                'odometer': {
                    'q3%': Q3_PERCENT_DEFAULT,
                    'q1%': Q1_PERCENT_DEFAULT
                },
                'year': {
                    'q3%': Q3_PERCENT_DEFAULT,
                    'q1%': Q1_PERCENT_DEFAULT
                }
            }
        },
        {
            'operation': 'queryFilter',
            'config': {
                'price > 0': {
                    'query': 'price > 0'
                },
                'odometer > 0': {
                    'query': 'odometer > 0'
                },
                'year > 1900': {
                    'query': 'year > 1900'
                }
            }
        },
        {
            'operation': 'fillna',
            'config': {
                'condition': {
                    'value': 'unknown'
                }
            }
        },
        {
            'operation': 'fillUsingMode',
            'config': {
                'Lookup&Fill Pass1': {
                    'lookupFields': ['manufacturer', 'cleaned_model', 'year'],
                    'fillFields': ['cylinders', 'size', 'type', 'drive']
                },
                'Lookup&Fill Pass2': {
                    'lookupFields': ['manufacturer', 'cleaned_model'],
                    'fillFields': ['cylinders', 'size', 'type', 'drive']
                },
                'Lookup&Fill Pass3': {
                    'lookupFields': ['manufacturer', 'type'],
                    'fillFields': ['size']
                }
            }
        },
        {
            'operation': 'fillna',
            'config': {
                'cylinders': {
                    'value': 'unknown'
                },
                'title_status': {
                    'value': 'unknown'
                },
                'transmission': {
                    'value': 'unknown'
                },
                'drive': {
                    'value': 'unknown'
                },
                'fuel': {
                    'value': 'unknown'
                },
                'size': {
                    'value': 'unknown'
                },
                'type': {
                    'value': 'unknown'
                }
            }
        }
    ],
    'labels': {
        'row_count': 'Row Count',
        'na_value_count': 'Empty Value #',
        'na_value_pct': 'Empty Value %',
        'unq_value_count': 'Unique Value Counts',
        'column': 'Feature'
    }
}
dataReportDf = addMetadataStatistics('raw', rawDf)
preProcessedDf, dataReportDf = preProcessData(dataPreparationRequest, rawDf, dataReportDf)
writeDataFrameDetails(data=preProcessedDf, fileNameSuffix=STEP02_DATA_PREPARATION, titleSuffix=': Data Cleaning & Preparation')
displayStats(dataPreparationRequest, dataReportDf)
writeString2File(json.dumps(dataPreparationRequest, indent=4), RESULT_FILE_PREFIX + STEP02_DATA_PREPARATION + '.request.json')
writeDataFrame2Excel(dataReportDf, STEP02_DATA_PREPARATION)

In [ ]:
def encodeAndAddCategoricalColumns(inputDf, targetField, categoricalFeatures):
    lowCardinalityFeatures = []
    highCardinalityFeatures = []

    renamedFields = {}
    for cf in categoricalFeatures:
        if cf != targetField:
            unqValues = len(inputDf[cf].value_counts())
            if unqValues > 255:
                renamedFields[cf] = 'encoded_' + cf
                highCardinalityFeatures.append(cf)
            else:
                lowCardinalityFeatures.append(cf)
            
    transformerSet = []
    if(len(lowCardinalityFeatures) > 0):
        transformerSet.append(('categorical_low_cardinality', OneHotEncoder(drop="if_binary", sparse_output=False), lowCardinalityFeatures))

    enhancedDf = inputDf
    
    # https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69
    if(len(highCardinalityFeatures) > 0):
        if USE_TARGET_ENCODER_FOR_HIGH_CARDINALTY:
            transformerSet.append(('categorical_high_cardinality', TargetEncoder(), highCardinalityFeatures))
        else:
            labelEncoder = LabelEncoder()
            for hcf in highCardinalityFeatures:
                enhancedDf['encoded_' + hcf] = labelEncoder.fit_transform(enhancedDf[hcf])

    
        
    columnTransformer = ColumnTransformer(transformerSet, verbose_feature_names_out=False)
    columnTransformer.set_output(transform='pandas')
    

    
    encodedDf = columnTransformer.fit_transform(enhancedDf, enhancedDf[targetField])
    encodedFeatureNames = columnTransformer.get_feature_names_out()
    encodedDf = encodedDf[encodedFeatureNames]
    if USE_TARGET_ENCODER_FOR_HIGH_CARDINALTY:
        encodedDf.rename(columns=renamedFields, inplace=True)
    enhancedDf = enhancedDf.join(encodedDf)
    return enhancedDf

def getRequestTemplate(targetField, numericFeaturesUsed, categoricalFeaturesUsed):
    requestTemplate = {
        'numericFeatures': numericFeaturesUsed,
        'categoricalFeatures': categoricalFeaturesUsed,
        'targetField': targetField,
        'featureTracker': {
            'features': {}
        }
    }
    return requestTemplate
    
def createRequestFromTemplate(requestTemplate, experimentName, estimator , parametersToTry = {}, extractImportantFeaturesFn = None):
    request = copy.deepcopy(requestTemplate)
    request['experimentName'] = experimentName
    request['estimator'] = estimator
    request['parametersToTry'] = parametersToTry
    request['extractImportantFeaturesFn'] = extractImportantFeaturesFn
    return request

numericFeatures = list(preProcessedDf.select_dtypes(include=[np.number]).columns.values)
categoricalFeatures = list(preProcessedDf.select_dtypes(exclude=[np.number]).columns.values)
print('Original Numeric Features: '+ str(numericFeatures))
print('Original Categorical Features: '+ str(categoricalFeatures))

####################
# Setup
####################
# Leave out 
# - VIN - No usuable information in this and no decisions based on vin based on domain knowledge
# - id - No usuable information in this and no decisions based on vin based on domain knowledge
# - paint_color - Lots of missing values that we cannot impute and reflects local preferences/tastes
# - model, cleaned_model - Too many values model does not return in reasonable time


numericFeaturesUsed = numericFeatures.copy()
categoricalFeaturesUsed = categoricalFeatures.copy()
for rf in IGNORE_FEATURES:
    if rf in numericFeaturesUsed:
        numericFeaturesUsed.remove(rf)
    if rf in categoricalFeaturesUsed:
        categoricalFeaturesUsed.remove(rf)

# Do transform for categorical now because baking into pipeline means sometings columns in training set aren't in test and vice versa
experimentDf = encodeAndAddCategoricalColumns(preProcessedDf, TARGET_FIELD, categoricalFeaturesUsed)

# Need to pull out any numeric fields like id that made it through
numericFeatures2Try = list(experimentDf.select_dtypes(include=[np.number]).columns.values)
for rf in IGNORE_FEATURES:
    if rf in numericFeatures2Try:
        numericFeatures2Try.remove(rf)

def drawCorrelation(data, name, minValueAbs = 0, drawPairPlot = False):
    corr = data.corr(numeric_only=True)
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    corr = corr.mask(mask | ((corr > -1 * minValueAbs) & (corr <= minValueAbs)), None)    
    corr.dropna(axis = 0, how = 'all', inplace = True)
    corr.dropna(axis = 1, how = 'all', inplace = True)
    height= max(corr.shape[0] * 100, 640) / 100
    width = max(corr.shape[1] * 100, 640) / 100
    sns.set (rc = {'figure.figsize':(width, height)})
    plt.clf()
    plt.figure()

    writeDataFrame2Excel(corr, STEP02_DATA_PREPARATION + name + '.correlation.');
    
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, cmap=cmap, annot=True, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

    title = f'Correlation Matrix for {name}'
    if minValueAbs != 0:
        title += f' (x <= -{minValueAbs} OR x >= {minValueAbs})'
    plt.title(title)
    plt.savefig(RESULT_FILE_PREFIX + STEP02_DATA_PREPARATION + name + '.data.heatmap.png')
    plt.show()

    # NOTE: Be careful very slow for large sets
    if True == drawPairPlot:
        plt.clf()
        plt.figure()
        sns.pairplot(data)
        plt.savefig(RESULT_FILE_PREFIX + STEP02_DATA_PREPARATION + name + '.data.pairplot.png')
        plt.show()



drawCorrelation(data = rawDf, name = 'raw')
drawCorrelation(data = preProcessedDf, name = 'preProcessed')
drawCorrelation(data = experimentDf, name = 'final', minValueAbs = 0.2)
print('Final Numeric Features: '+ str(numericFeatures2Try))
experimentDf = experimentDf[numericFeatures2Try]

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
def runFeatureSelectionAnalysis(request, inputDf, modelReportDf, featureImportanceReportDf):
    experimentName = request['experimentName']
    numericFeatures = request['numericFeatures']
    categoricalFeatures = request['categoricalFeatures']
    targetField = request['targetField']
    estimator = request['estimator']
    parametersToTry = request['parametersToTry']
    featureTracker = request['featureTracker']
    extractImportantFeaturesFn = request['extractImportantFeaturesFn']

    # Make copy of data
    experimentDf = inputDf

    # Just in case remove target field from feeatures
    if targetField in numericFeatures:
        numericFeatures.remove(targetField)
        
    # Build train + dev set
    X = experimentDf.drop(targetField, axis=1);
    y = experimentDf[targetField]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

    gridSearchCV = GridSearchCV(estimator = estimator, 
                                        param_grid = parametersToTry, 
                                        scoring = "neg_mean_squared_error",
                                        cv = 5)
    gridSearchCV.fit(X_train, y_train)
    
    # Get best performer
    bestEstimator = gridSearchCV.best_estimator_
    bestScore = gridSearchCV.best_score_
    bestParameters = json.dumps(gridSearchCV.best_params_, indent=2)
    #print("CV Results" + str(gridSearchCV.cv_results_))
    
    
    # Display processing pipeline
    display(gridSearchCV)

    # Build cofficient importances
    coefDf = extractImportantFeaturesFn(bestEstimator)
    coefDf['Model'] = experimentName
    featureImportanceReportDf = pd.concat([featureImportanceReportDf, coefDf])

    # Get predictions for train and test sets
    y_predict_train = bestEstimator.predict(X_train)
    y_predict_test = bestEstimator.predict(X_test)
    
    # Add to report
    modelReportDf.loc[len(modelReportDf)] = [
        experimentName, 
        ', '.join(categoricalFeatures), 
        ', '.join(numericFeatures),
        bestParameters,
        round(metrics.mean_squared_error(y_predict_train, y_train), 2), 
        round(metrics.mean_squared_error(y_predict_test, y_test), 2), 
        round(metrics.mean_absolute_error(y_predict_train, y_train), 2), 
        round(metrics.mean_absolute_error(y_predict_test, y_test), 2), 
        round(metrics.r2_score(y_predict_train, y_train), 2), 
        round(metrics.r2_score(y_predict_test, y_test), 2) 
    ]

    # Track feature performance across runs
    for row in coefDf.itertuples():
        field = row.index
        if field not in featureTracker:
            featureTracker[field] = {
                'count': 0,
                'score': 0.0
            }
        featureTracker[field]['count'] = featureTracker[field]['count'] + 1
    
    return modelReportDf, featureImportanceReportDf
    

# Setup base request teamplate
REQUEST_TEMPLATE = getRequestTemplate(TARGET_FIELD, numericFeaturesUsed, categoricalFeaturesUsed)

# Setup processing reporting dataframes
modelReportDf = pd.DataFrame(columns=[
    'Model', 
    'Categorical Features', 
    'Numerical Features', 
    'Best Parameters', 
    'Train MSE', 
    'Test MSE',
    'Train MAE', 
    'Test MAE',
    'Train R2', 
    'Test R2'
])
featureImportanceReportDf = pd.DataFrame(columns=['Model'])

####################
# Model 1: Ridge
####################
if MODEL1_ACTIVE:
    pipeline1 = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge())
    ], )
    parameters1 = {
        'regressor__alpha' : ALPHA_RANGE
    }
    def getImportantFeatures1(bestEstimator):
        feature_names = bestEstimator[:-1].get_feature_names_out()
        bestFeatureCoefficients = bestEstimator[-1].coef_
        coefDf = pd.DataFrame(
            bestFeatureCoefficients,
            columns=["coefficient"],
            index=feature_names
        ).reset_index()
        return coefDf
    
    request1 = createRequestFromTemplate(REQUEST_TEMPLATE, 'Ridge', pipeline1, parameters1, getImportantFeatures1)
    modelReportDf, featureImportanceReportDf = runFeatureSelectionAnalysis(request1, experimentDf, modelReportDf, featureImportanceReportDf)

####################
# Model 2: Lasso
####################
if MODEL2_ACTIVE:
    pipeline2 = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Lasso())
    ], )
    parameters2 = {
        'regressor__alpha' : ALPHA_RANGE
    }
    def getImportantFeatures2(bestEstimator):
        feature_names = bestEstimator[:-1].get_feature_names_out()
        bestFeatureCoefficients = bestEstimator[-1].coef_
        coefDf = pd.DataFrame(
            bestFeatureCoefficients,
            columns=["coefficient"],
            index=feature_names
        ).reset_index()
        return coefDf
    
    request2 = createRequestFromTemplate(REQUEST_TEMPLATE, 'Lasso', pipeline2, parameters2, getImportantFeatures2)
    modelReportDf, featureImportanceReportDf = runFeatureSelectionAnalysis(request2, experimentDf, modelReportDf, featureImportanceReportDf)

####################
# Model 3: ElasticNet
####################
if MODEL3_ACTIVE:
    pipeline3 = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', ElasticNet())
    ], )
    parameters3 = {
        'regressor__alpha' : ALPHA_RANGE
    }
    def getImportantFeatures3(bestEstimator):
        feature_names = bestEstimator[:-1].get_feature_names_out()
        bestFeatureCoefficients = bestEstimator[-1].coef_
        coefDf = pd.DataFrame(
            bestFeatureCoefficients,
            columns=["coefficient"],
            index=feature_names
        ).reset_index()
        return coefDf
    
    request3 = createRequestFromTemplate(REQUEST_TEMPLATE, 'ElasticNet', pipeline3, parameters3, getImportantFeatures3)
    modelReportDf, featureImportanceReportDf = runFeatureSelectionAnalysis(request3, experimentDf, modelReportDf, featureImportanceReportDf)

####################
# Model 4: RandomForestRegressor
####################
if MODEL4_ACTIVE:
    pipeline4 = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', RandomForestRegressor())
    ], )
    parameters4 = {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 1, 10, 20]
    }
    def getImportantFeatures4(bestEstimator):
        feature_names = bestEstimator[:-1].get_feature_names_out()
        bestFeatureCoefficients = bestEstimator[-1].feature_importances_
        coefDf = pd.DataFrame(
            bestFeatureCoefficients,
            columns=["coefficient"],
            index=feature_names
        ).reset_index()
        return coefDf
    
    request4 = createRequestFromTemplate(REQUEST_TEMPLATE, 'RandomForestRegressor', pipeline4, parameters4, getImportantFeatures4)
    modelReportDf, featureImportanceReportDf = runFeatureSelectionAnalysis(request4, experimentDf, modelReportDf, featureImportanceReportDf)

writeDataFrame2Excel(modelReportDf, STEP03_MODELING + 'model_report.')
writeDataFrame2Excel(featureImportanceReportDf, STEP03_MODELING + 'feature_report.')
dataFrame2Image(modelReportDf, RESULT_FILE_PREFIX + STEP03_MODELING + 'performance.png', height_per_row=10)

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [ ]:
def showFeatureEvaluationBarPlots(request, subPlots, pathSuffix, subPlotCols = 3, width=1200, height=400):
    filePrefix = RESULT_FILE_PREFIX + STEP04_EVALUATION
    labels = request['labels']
    numSubPlots = len(subPlots)
    
    subPlotRows = math.ceil( numSubPlots / subPlotCols)
    palette='pastel'
    barWidth=0.8
    widthInches = width/100
    heightInches = height/100
    plt.clf()
    fig, ax = plt.subplots(subPlotRows, subPlotCols, figsize=(widthInches, heightInches))
    subPlotRow = 0
    subPlotCol = 0
    for subPlot in subPlots:
        data = subPlot['data']
        xColumn = subPlot['xColumn']
        yColumn = subPlot['yColumn']
        xLogScale = False
        if 'xLogScale' in subPlot:
            xLogScale = subPlot['xLogScale']
        yLogScale = False
        if 'yLogScale' in subPlot:
            yLogScale = subPlot['yLogScale']
        xAxesLabel = labels[xColumn]
        if xLogScale:
            xAxesLabel += ' (Log Scale)'
        yAxesLabel = labels[yColumn]
        if yLogScale:
            yAxesLabel += ' (Log Scale)'
        title = yAxesLabel + ' vs ' + xAxesLabel
        if 'title' in subPlot:
            title = subPlot['title']
        
        width=min(data.shape[0]/2, 1200)
        height=8
        legendLocation='lower left'
        bboxAnchor=(0, -0.5)
        
        axSub = sns.barplot(ax=ax[subPlotRow, subPlotCol],data=data, x=xColumn, y=yColumn,
                         palette=palette, width=barWidth)
        axSub.set_title(title)
        axSub.set(xlabel=xAxesLabel, ylabel=yAxesLabel)
        for bars_group in axSub.containers: 
            axSub.bar_label(bars_group, padding=5, fontsize=6, label_type='center')
        if yLogScale:
            plt.yscale('log')
        if xLogScale:
            plt.xscale('log')
        #sns.move_legend(ax, legendLocation, bbox_to_anchor=bboxAnchor)
        #ax.figure.set_size_inches(width,height)

        subPlotCol +=1
        if subPlotCol % subPlotCols == 0:
            subPlotRow += 1
            subPlotCol = 0

    # Get rid of extra plots
    if subPlotCol % subPlotCols == 0:
            subPlotRow += 1
            subPlotCol = 0
    
    for i in range(subPlotCol, subPlotCols):
        fig.delaxes(ax[subPlotRow][i])
        
    plt.savefig(filePrefix + pathSuffix, bbox_inches='tight')
    plt.show()
    
def graphFeatureCoefficients(request, modelPerformanceReportDf, featureCoefficientReportDf, categoryFeatureSets):
    filePrefix = RESULT_FILE_PREFIX + STEP04_EVALUATION

    # Sort we can find best performer base on TestMSE
    modelPerfDf = modelPerformanceReportDf.sort_values('Test MSE', ascending=True)
    bestPerformer = modelPerfDf.iloc[0]['Model']
    

    # All coefficients across all models. Copy so we can change
    featureImportanceDf = featureCoefficientReportDf
    featureImportanceDf['coefficient_abs'] = featureImportanceDf['coefficient'].abs()
    featureImportanceDf = featureImportanceDf.sort_values('coefficient_abs', ascending=False)

    # Show all model performances
    xColumn = 'coefficient'
    colorColumn = 'Model'
    yColumn = 'index'
    fileSuffix = 'coefficient.png'
    showBarPlot(request=request, data=featureImportanceDf, xColumn=xColumn, yColumn=yColumn, colorColumn=colorColumn, 
                path=filePrefix + fileSuffix, height=(featureImportanceDf.shape[0] * 30), width=2400)


    # Show winner
    bestModelFeatureImportanceDf = featureImportanceDf.loc[featureImportanceDf['Model'] == bestPerformer]
    subPlotsPositiveFeatures = []
    subPlotsNegativeFeatures = []
    xColumn = 'coefficient_abs'
    for key, value in categoryFeatureSets.items():
        features = value
        for f in features:
            # Copy so we can add more columns
            featureDf = bestModelFeatureImportanceDf.loc[bestModelFeatureImportanceDf['index'].str.startswith(f)].copy()
            featureDf['index'] = featureDf['index'].str.replace(f + '_', '')
            posDf = featureDf.loc[featureDf['coefficient'] > 0]
            negDf = featureDf.loc[featureDf['coefficient'] <= 0]
            subPlotsPositiveFeatures.append(
                {
                    'data': posDf,
                    'title': f'Positive {f} Feature Values',
                    'xColumn': xColumn,
                    'xLogScale': True,
                    'yColumn': yColumn
                }
            )
            subPlotsNegativeFeatures.append(
                {
                    'data': negDf,
                    'title': f'Negative {f} Feature Values',
                    'xColumn': xColumn,
                    'xLogScale': True,
                    'yColumn': yColumn
                }
            )
    showFeatureEvaluationBarPlots(request=request, subPlots=subPlotsPositiveFeatures, pathSuffix='pos.coeff.png', width=2800, height=2400)
    showFeatureEvaluationBarPlots(request=request, subPlots=subPlotsNegativeFeatures, pathSuffix='neg.coeff.png', width=2800, height=2400)

request = {
    'labels': {
        'coefficient': 'Coefficent',
        'coefficient_abs': 'Coefficent',
        'feature_value': 'Feature Value',
        'index': 'Feature'
    }
}
featureSets = {
    'vehicle': ['manufacturer','type', 'size', 'drive', 'transmission', 'cylinders', 'fuel'],
    'ownership': ['condition', 'title_status'],
    'location': ['state']
}
graphFeatureCoefficients(request, modelReportDf, featureImportanceReportDf, featureSets)

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.